<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# Lab 2 - Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.db

'Connected: @movies.db'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.db
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [4]:
%%sql
SELECT *
FROM movies

 * sqlite:///movies.db
Done.


imdb_key,movie_name,production_year,running_time
tt0111161,The Shawshank Redemption,1994,144
tt0816692,Interstellar,2014,169
tt0892769,How to Train your Dragon,2010,98
tt2084970,The Imitation Game,2014,114
tt1201607,Harry Potter and the Deathly Hallows: Part 2,2011,130
tt0167260,Lord of the Rings: The Return of the King,2003,201


+ Show the performance dates for one of the movies.

In [5]:
%%sql
SELECT  start_time, start_date
FROM    performances
WHERE   imdb_key = "tt0111161"

 * sqlite:///movies.db
Done.


start_time,start_date
19:30,2019-02-16
20:30,2019-02-16
19:30,2019-02-14


+ Show all data concerning performances at a given theatere
  on a given date.

In [6]:
%%sql
SELECT  *
FROM    theaters
JOIN    performances
USING   (theater_name)
WHERE   start_date = "2019-02-16" AND theater_name = "Filmstaden Lund"

 * sqlite:///movies.db
Done.


theater_name,capacity,performance_id,start_time,start_date,imdb_key
Filmstaden Lund,237,123,19:30,2019-02-16,tt0111161
Filmstaden Lund,237,234,20:30,2019-02-16,tt0111161


+ List all customers

In [7]:
%%sql
SELECT   *
FROM     customers

 * sqlite:///movies.db
Done.


username,full_name,password
Bertil45,Bert Jansson,1234
Pelle1337,Pär Nilsson,password
Kurtinator,Kurt Kurtsson,kurtärbäst
Nilzzz,Nils Bo,zzzlin
WalterB,Walter Andersson,princess


+ List all tickets

In [11]:
%%sql
SELECT   *
FROM     tickets

 * sqlite:///movies.db
Done.


t_id,username,performance_id
9462196aec40825c047262926cd57449,WalterB,123
391613f44064654f50b161c37eb68ca2,WalterB,123
cd0bc55ce9390741bf8e4277a0518d3b,Nilzzz,234
072924de37540cdee8da92aa6c2bfba5,Kurtinator,567
059d15ee640f7cdef7b68cce57cef54c,Kurtinator,567


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [10]:
%%sql
INSERT 
INTO tickets (performance_id, username) 
VALUES       ("567", "Kurtinator")

 * sqlite:///movies.db
1 rows affected.


[]

In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2019), we can't do that, instead
  we can the following idea: each row in a SQLite3 table has
  a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries, unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [12]:
%%sql
SELECT t_id
FROM   tickets
WHERE  rowid = last_insert_rowid()

 * sqlite:///movies.db
Done.


t_id
059d15ee640f7cdef7b68cce57cef54c


+ Try to insert two movie theaters with the same name (this
  should fail).

In [15]:
%%sql
INSERT 
INTO theaters 
VALUES        ("Filmstaden Lund", 237)

 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name [SQL: 'INSERT \nINTO theaters \nVALUES        ("Filmstaden Lund", 237)'] (Background on this error at: http://sqlalche.me/e/gkpj)

+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [16]:
%%sql
INSERT 
INTO          performances 
VALUES        ("123", "19:30", "2019-02-16", "Filmstaden Lunfd", "tt0111161")

 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: performances.performance_id [SQL: 'INSERT \nINTO          performances \nVALUES        ("123", "19:30", "2019-02-16", "Filmstaden Lunfd", "tt0111161")'] (Background on this error at: http://sqlalche.me/e/gkpj)

+ Create a ticket where either the user or the performance
  doesnâ€™t exist.

In [17]:
%%sql
INSERT 
INTO tickets   (performance_id, username)
VALUES         ("123", "f")

 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: 'INSERT \nINTO tickets   (performance_id, username)\nVALUES         ("123", "f")'] (Background on this error at: http://sqlalche.me/e/gkpj)